<a href="https://colab.research.google.com/github/ignat24/Python/blob/master/Argumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy
import math
from tensorflow import keras

# from keras.datasets import mnist
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils,Sequence
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import albumentations as A

# Устанавливаем seed для повторяемости результатов
numpy.random.seed(42)

# Размер изображения
img_rows, img_cols = 32, 32

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Преобразование размерности изображений
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)


# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train,10)
Y_test = np_utils.to_categorical(y_test,10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

AUGM = A.Compose([    
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.2, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])

class DataGenerator(Sequence):
 
    def __init__(self, X_data , y_data, batch_size, for_train, augm, shuffle = True):
        self.X_data = X_data
        self.y_data = y_data 
        self.batch_size = batch_size
        self.for_train = for_train
        self.augm = augm
        self.shuffle = shuffle
        self.n = 0
        self.list_IDs = numpy.arange(len(self.X_data))
        self.on_epoch_end()
        
    def __next__(self):
        # Get one batch of data
        data = self.__getitem__(self.n)
        # Batch index
        self.n += 1
        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end()
            self.n = 0        
        return data

    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]        
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(list_IDs_temp)
        
        if self.for_train:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X
        
    def on_epoch_end(self):
        self.indexes = numpy.arange(len(self.X_data))
        if self.shuffle: 
            numpy.random.shuffle(self.indexes)    
            
    def _generate_x(self, list_IDs_temp):
        X = numpy.zeros((self.batch_size, 32, 32, 3), dtype=numpy.float32)
        for i, ID in enumerate(list_IDs_temp):
            X[i] = self.X_data[ID]
            if self.augm:                
                img_augm = AUGM(image=X[i])
                X[i] = img_augm["image"]                
        return X
    
    def _generate_y(self, list_IDs_temp):
        #y = np.empty(self.batch_size)        
        y = numpy.zeros((self.batch_size, 10), dtype=numpy.float32)
        for i, ID in enumerate(list_IDs_temp):            
            y[i] = self.y_data[ID]            
        return y

MODELS_COUNT=5

y_pred=None
for i in range(MODELS_COUNT):
  print("Model #",i)


  # Создаем последовательную модель
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(32,32,3),padding = "same"))
  model.add(Conv2D(64, kernel_size=(4, 4),
                  activation='relu',
                   padding = "same"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu',padding = "same"))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(Dropout(0.2)) 
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))

  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  # print(model.summary())

  # checkpoint_filepath = 'model_' + str(i) + '.h5'
  # model_checkpoint_callback = ModelCheckpoint(
  #     filepath=checkpoint_filepath,
  #     save_weights_only=True,
  #     montior='val_accuracy',
  #     mode='max',
  #     save_best_only=True)
  train_generator = DataGenerator(X_train, Y_train, batch_size = 200, 
                                    for_train=True, augm = True, shuffle=True)
  
  val_generator = DataGenerator(X_val, Y_val, batch_size = 200, 
                                    for_train=True, augm = False, shuffle=False)
  
  history = model.fit_generator(train_generator,
                                validation_data=val_generator,
                                epochs=20, verbose=1) 
  
  # model.fit(X_train,Y_train,batch_size=128, epochs=25, validation_split=0.1, verbose=2)

  # model.load_weights(checkpoint_filepath)

  scores=model.evaluate(X_test, Y_test, verbose=0)
  print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

  if y_pred is None:
    y_pred = model.predict(X_test)
  else:
    y_pred +=model.predict(X_test)

y_pred /= MODELS_COUNT
pred = numpy.argmax(y_pred, axis=1)
acc_test = accuracy_score(y_test,pred)
# print("acc_test: ()".format(acc_test))
print("Final accuracy: {}%".format(round(acc_test*100,2)))



Model # 0
Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 9s 40ms/step - loss: 2.0312 - accuracy: 0.2557 - val_loss: 1.4285 - val_accuracy: 0.4754
Epoch 2/20
200/200 [==============================] - 8s 40ms/step - loss: 1.5123 - accuracy: 0.4507 - val_loss: 1.2002 - val_accuracy: 0.5725
Epoch 3/20
200/200 [==============================] - 8s 40ms/step - loss: 1.3367 - accuracy: 0.5235 - val_loss: 1.0951 - val_accuracy: 0.6162
Epoch 4/20
200/200 [==============================] - 8s 40ms/step - loss: 1.2319 - accuracy: 0.5610 - val_loss: 0.9803 - val_accuracy: 0.6472
Epoch 5/20
200/200 [==============================] - 8s 39ms/step - loss: 1.1561 - accuracy: 0.5910 - val_loss: 0.9260 - val_accuracy: 0.6696
Epoch 6/20
200/200 [==============================] - 8s 39ms/step - loss: 1.0819 - accuracy: 0.6192 - val_loss: 0.8047 - val_accuracy: 0.7180
Epoch 7/20
200/200 [==============================] - 8s 39ms/step - loss: 1.0134 - accuracy: 0.6450 - val_loss: 0.8058 - val_accuracy: 0.7217
Epoch 8/20